In [11]:
! pip install torchsummary

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from datetime import datetime
from matplotlib import pyplot as plt
from time import time
from torch.utils.data import DataLoader
from torchsummary import summary

torch.__version__

You should consider upgrading via the 'pip install --upgrade pip' command.


'1.7.1'

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [13]:
! pip install wandb -qqq
import wandb
wandb.login()

PROJECT = 'INCEPTION_{}'.format(datetime.now().strftime('%y%m%d'))
PROJECT

'INCEPTION_201230'

In [3]:
batch_size = 64

train_dataset = datasets.MNIST(root='data/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='data/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=True)

7.2%

100.1%

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


2.0%5%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


180.4%/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:439: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


In [ ]:
def accuracy(model, data_loader):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(data_loader):
            x, y = x.to(device), y.to(device)
            y_pred = model(x).argmax(1)
            num_correct += (y == y_pred).sum()
            num_samples += y.size(0)
    acc = num_correct / num_samples
    return acc

def train(run_name, model, data_loader, num_epochs=2, learning_rate=1e-3):
    name = '{}_{}'.format(run_name, datetime.now().strftime('%H%M%S'))
    wandb.init(project=PROJECT, name=name)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fun = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        model.train()
        start_time = time()
        for batch_idx, (x, y) in enumerate(data_loader):
            x, y = x.to(device), y.to(device)
            scores = model(x)
            loss = loss_fun(scores, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            wandb.log({'loss': loss})
        print(f'epoch {epoch}, {time() - start_time:.1f}s: {accuracy(model, data_loader):.1%}')

$$O=\frac{I+2P-K}{S}+1$$

In [17]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0))
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)
        self.to(device)
        
    def forward(self, x): 
        x = self.relu(self.conv1(x)) 
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x)) 
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    
model = LeNet()
summary(model, (1, 28, 28))
train('LeNet', model, train_loader)
print(f'test: {accuracy(model, test_loader):.1%}')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              ReLU-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              ReLU-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
            Conv2d-7            [-1, 120, 1, 1]          48,120
              ReLU-8            [-1, 120, 1, 1]               0
            Linear-9                   [-1, 84]          10,164
             ReLU-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

wandb: ERROR Problem finishing run
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 1215, in _atexit_cleanup
    self._on_finish()
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 1366, in _on_finish
    ret = self._wait_for_finish()
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 1329, in _wait_for_finish
    self._on_finish_progress(pusher_stats, done)
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 1318, in _on_finish_progress
    self._pusher_print_status(progress, done=done)
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 1294, in _pusher_print_status
    self._jupyter_progress.close()
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/lib/ipython.py", line 79, in close
    self._widget.close()
  File "/usr/local/lib/python3.6/dist-packages/ipywidgets/widgets/widget.py", line 469, in close
    self.c

epoch 0, 10.9s: 95.5%
epoch 1, 10.5s: 97.0%
test: 97.1%


In [16]:
class VGG(nn.Module):
    def __init__(self, architecture, in_channels=3, in_size=(224, 224)):
        super(VGG, self).__init__()
        self.cnv = self.create_conv_layers(in_channels, architecture[0])
        out_size = torch.tensor(in_size) // 2 ** architecture[0].count('M')
        self.fcs = self.create_fcs_layers(architecture[0][-2] * out_size.prod(), architecture[1])
        self.to(device)
        
    def forward(self, x):
        x = self.cnv(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x
        
    def create_conv_layers(self, in_channels, architecture):
        layers = []
        for out in architecture:
            if type(out) == int:
                layers += [
                    nn.Conv2d(in_channels, out, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.BatchNorm2d(out),
                    nn.ReLU()
                ]
                in_channels = out
            elif out == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]
        return nn.Sequential(*layers)
    
    def create_fcs_layers(self, in_channels, architecture):
        layers = []
        for out in architecture[:-1]:
            layers += [
                nn.Linear(in_channels, out),
                nn.ReLU(),
                nn.Dropout(p=0.5)
            ]
            in_channels = out
        layers += [nn.Linear(in_channels, architecture[-1])]
        return nn.Sequential(*layers)
        
tiny = [
    [8, 8, 'M', 16, 16, 'M'],
    [64, 10]
]

model = VGG(tiny, in_channels=1, in_size=(28, 28))
summary(model, (1, 28, 28))
train('VGG', model, train_loader)
print(f'test: {accuracy(model, test_loader):.1%}')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
              ReLU-3            [-1, 8, 28, 28]               0
            Conv2d-4            [-1, 8, 28, 28]             584
       BatchNorm2d-5            [-1, 8, 28, 28]              16
              ReLU-6            [-1, 8, 28, 28]               0
         MaxPool2d-7            [-1, 8, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           1,168
       BatchNorm2d-9           [-1, 16, 14, 14]              32
             ReLU-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           2,320
      BatchNorm2d-12           [-1, 16, 14, 14]              32
             ReLU-13           [-1, 16, 14, 14]               0
        MaxPool2d-14             [-1, 1

epoch 0, 13.4s: 98.3%
epoch 1, 13.3s: 98.7%
test: 98.7%


![](https://miro.medium.com/max/700/1*U_McJnp7Fnif-lw9iIC5Bw.png)

In [72]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(ConvBlock, self).__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        
    def forward(self, x):
        return self.relu(self.batchnorm(self.conv(x)))
    
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1_pool):
        super(InceptionBlock, self).__init__()
        self.branch1 = ConvBlock(in_channels, out_1x1, kernel_size=1)
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, red_3x3, kernel_size=1),
            ConvBlock(red_3x3, out_3x3, kernel_size=3, padding=1)
        )
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, red_5x5, kernel_size=1),
            ConvBlock(red_5x5, out_5x5, kernel_size=5, padding=2),
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels, out_1x1_pool, kernel_size=1)
        )
        
    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)
    
class GoogLeNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=10000):
        super(GoogLeNet, self).__init__()
        self.conv1 = ConvBlock(in_channels, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = ConvBlock(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)
        self.avgpool = nn.AvgPool2d(kernel_size=7)
        self.dropout = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(1024, num_classes)
        self.to(device)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc1(x)
        return x
        
if __name__ == '__main__':
    x = torch.randn(3, 3, 224, 224).to(device)
    model = GoogLeNet()
    print(model(x).shape)
    # summary(model, (3, 224, 224))
        
        
    

torch.Size([3, 10000])


In [82]:
class TinyInception(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(TinyInception, self).__init__()
        self.conv1 = ConvBlock(in_channels, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = ConvBlock(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)
        # self.avgpool = nn.AvgPool2d(kernel_size=3)
        self.dropout = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(1024, num_classes)
        self.to(device)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        # x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc1(x)
        return x
    

# x = torch.randn(3, 1, 28, 28).to(device)
# model = TinyInception()
# print(model(x).shape)

model =  TinyInception()
summary(model, (1, 28, 28))
train('TinyInception', model, train_loader)
print(f'test: {accuracy(model, test_loader):.1%}')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]           3,200
       BatchNorm2d-2           [-1, 64, 14, 14]             128
              ReLU-3           [-1, 64, 14, 14]               0
         ConvBlock-4           [-1, 64, 14, 14]               0
         MaxPool2d-5             [-1, 64, 7, 7]               0
            Conv2d-6            [-1, 192, 7, 7]         110,784
       BatchNorm2d-7            [-1, 192, 7, 7]             384
              ReLU-8            [-1, 192, 7, 7]               0
         ConvBlock-9            [-1, 192, 7, 7]               0
        MaxPool2d-10            [-1, 192, 4, 4]               0
           Conv2d-11             [-1, 64, 4, 4]          12,352
      BatchNorm2d-12             [-1, 64, 4, 4]             128
             ReLU-13             [-1, 64, 4, 4]               0
        ConvBlock-14             [-1, 6

wandb: ERROR Problem finishing run
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 1215, in _atexit_cleanup
    self._on_finish()
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 1366, in _on_finish
    ret = self._wait_for_finish()
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 1329, in _wait_for_finish
    self._on_finish_progress(pusher_stats, done)
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 1318, in _on_finish_progress
    self._pusher_print_status(progress, done=done)
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 1294, in _pusher_print_status
    self._jupyter_progress.close()
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/lib/ipython.py", line 79, in close
    self._widget.close()
  File "/usr/local/lib/python3.6/dist-packages/ipywidgets/widgets/widget.py", line 469, in close
    self.c

epoch 0, 69.4s: 97.4%
epoch 1, 69.7s: 97.6%
test: 97.6%
